In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
import warnings
import numpy as np
warnings.filterwarnings('ignore')
import time
from models import Informer, Autoformer, Transformer, DLinear, Linear, NLinear, PatchTST
import argparse
from torch import optim
import torch.nn as nn

In [2]:
time_steps = 3000
num_features = 10
pred_len = 60
label_len = 40

In [3]:
data_x = np.random.uniform(low=1, high=100, size=(time_steps, num_features))
data_y = data_x


In [4]:
data_x.shape, data_y.shape

((3000, 10), (3000, 10))

In [5]:
device = torch.device('mps')

In [6]:
class DummyPretrainDataset(Dataset):
    def __init__(self, data_x, data_y, num_features, time_steps=15, seq_len=96, pred_len=96, label_len=48, bs=5):
        self.num_features = num_features
        self.time_steps = time_steps
        self.seq_len = seq_len
        self.pred_len = pred_len
        self.label_len = label_len
        self.batch_size = bs
        self.prebatch_len = seq_len + bs  - 1
        self.data_x = data_x
        self.data_y = data_y

    def __len__(self):
        return (self.time_steps - self.seq_len - pred_len + 1) // self.batch_size 

    def __getitem__(self, idx):
        s_begin = self.batch_size * idx
        s_end = s_begin + self.prebatch_len 
        # r_begin = s_end - 1 # - self.label_len
        # r_end = r_begin + 1 # + self.pred_len  + self.label_len 
        #regression
        # r_begin = s_begin + self.seq_len - 1
        # r_end = r_begin + self.batch_size
        
        r_begin = s_begin + self.seq_len - self.label_len -1
        r_end = r_begin + self.pred_len + self.label_len + self.batch_size - 1
        # print(s_begin, s_end, r_begin, r_end)

        seq_x = self.data_x[s_begin:s_end]
        seq_y = self.data_y[r_begin:r_end]
        return torch.Tensor(seq_x).to(device), torch.Tensor(seq_y).to(device)

In [7]:
dset = DummyPretrainDataset(data_x, data_y, num_features, time_steps, seq_len=500, pred_len=pred_len, label_len=label_len, bs=100)

In [8]:
dset[0][0].shape, dset[0][1].shape

(torch.Size([599, 10]), torch.Size([199, 10]))

In [9]:
len(dset)

24

In [8]:
class SlidingWindowView:
    def __init__(self, window_size, stride, pred_len, label_len):
        self.window_size = window_size
        self.stride = stride
        self.pred_len = pred_len
        self.label_len = label_len
    def slide_collate_fn(self, batch):
        return batch[0].unfold(0, self.window_size, self.stride).transpose(1,2), batch[1].unfold(0, self.pred_len+self.label_len, self.stride).transpose(1,2)

In [9]:
sw = SlidingWindowView(500, 1, pred_len, label_len)
w_dl = DataLoader(dset, batch_size=None, collate_fn=sw.slide_collate_fn, pin_memory=False)

In [6]:
class DummyPretrainStackDataset(Dataset):
    def __init__(self, data_x, data_y, num_features, time_steps=15, seq_len=6, pred_len=8, label_len=8, bs=5):
        self.num_features = num_features
        self.time_steps = time_steps
        self.seq_len = seq_len
        self.pred_len = pred_len
        self.label_len = label_len
        # self.batch_size = bs
        # self.prebatch_len = seq_len + bs - 1
        self.data_x = data_x
        self.data_y = data_y

    def __len__(self):
        return (self.time_steps - self.seq_len + 1 - self.pred_len)  # // self.batch_size 

    def __getitem__(self, idx):
        s_begin = idx
        s_end = s_begin + self.seq_len
        r_begin = s_end - self.label_len
        r_end = r_begin  + self.pred_len  + self.label_len 

        seq_x = self.data_x[s_begin:s_end]
        seq_y = self.data_y[r_begin:r_end]
        return torch.Tensor(seq_x), torch.Tensor(seq_y)
# b1[0].float(), b1[0][:,:,:4].float(), dec_inp, b1[1][:,:,:4].float()

In [7]:
sset = DummyPretrainStackDataset(data_x, data_y, num_features, time_steps, seq_len=500, pred_len=60, label_len=40)

In [11]:
s_dl = DataLoader(sset, batch_size=100, shuffle=False, drop_last=True)

In [12]:
len(s_dl)

24

In [9]:

parser = argparse.ArgumentParser(description='Autoformer & Transformer family for Time Series Forecasting')

# random seed
parser.add_argument('--random_seed', type=int, default=2021, help='random seed')

# # basic config
# parser.add_argument('--is_training', type=int, required=True, default=1, help='status')
# parser.add_argument('--model_id', type=str, required=True, default='test', help='model id')
# parser.add_argument('--model', type=str, required=True, default='Autoformer',
#                     help='model name, options: [Autoformer, Informer, Transformer]')

# # data loader
# parser.add_argument('--data', type=str, required=True, default='ETTm1', help='dataset type')
parser.add_argument('--root_path', type=str, default='./data/ETT/', help='root path of the data file')
parser.add_argument('--data_path', type=str, default='ETTh1.csv', help='data file')
parser.add_argument('--features', type=str, default='M',
                    help='forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate')
parser.add_argument('--target', type=str, default='OT', help='target feature in S or MS task')
parser.add_argument('--freq', type=str, default='h',
                    help='freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h')
parser.add_argument('--checkpoints', type=str, default='./checkpoints/', help='location of model checkpoints')

# forecasting task
parser.add_argument('--seq_len', type=int, default=500, help='input sequence length')
parser.add_argument('--label_len', type=int, default=label_len, help='start token length')
parser.add_argument('--pred_len', type=int, default=pred_len, help='prediction sequence length')


# DLinear
#parser.add_argument('--individual', action='store_true', default=False, help='DLinear: a linear layer for each variate(channel) individually')

# PatchTST
parser.add_argument('--fc_dropout', type=float, default=0.05, help='fully connected dropout')
parser.add_argument('--head_dropout', type=float, default=0.0, help='head dropout')
parser.add_argument('--patch_len', type=int, default=16, help='patch length')
parser.add_argument('--stride', type=int, default=8, help='stride')
parser.add_argument('--padding_patch', default='end', help='None: None; end: padding on the end')
parser.add_argument('--revin', type=int, default=1, help='RevIN; True 1 False 0')
parser.add_argument('--affine', type=int, default=0, help='RevIN-affine; True 1 False 0')
parser.add_argument('--subtract_last', type=int, default=0, help='0: subtract mean; 1: subtract last')
parser.add_argument('--decomposition', type=int, default=0, help='decomposition; True 1 False 0')
parser.add_argument('--kernel_size', type=int, default=25, help='decomposition-kernel')
parser.add_argument('--individual', type=int, default=0, help='individual head; True 1 False 0')

# Formers 
parser.add_argument('--embed_type', type=int, default=0, help='0: default 1: value embedding + temporal embedding + positional embedding 2: value embedding + temporal embedding 3: value embedding + positional embedding 4: value embedding')
parser.add_argument('--enc_in', type=int, default=num_features, help='encoder input size') # DLinear with --individual, use this hyperparameter as the number of channels
parser.add_argument('--dec_in', type=int, default=num_features, help='decoder input size')
parser.add_argument('--c_out', type=int, default=num_features, help='output size')
parser.add_argument('--d_model', type=int, default=512, help='dimension of model')
parser.add_argument('--n_heads', type=int, default=8, help='num of heads')
parser.add_argument('--e_layers', type=int, default=2, help='num of encoder layers')
parser.add_argument('--d_layers', type=int, default=1, help='num of decoder layers')
parser.add_argument('--d_ff', type=int, default=2048, help='dimension of fcn')
parser.add_argument('--moving_avg', type=int, default=25, help='window size of moving average')
parser.add_argument('--factor', type=int, default=1, help='attn factor')
parser.add_argument('--distil', action='store_false',
                    help='whether to use distilling in encoder, using this argument means not using distilling',
                    default=True)
parser.add_argument('--dropout', type=float, default=0.05, help='dropout')
parser.add_argument('--embed', type=str, default='timeF',
                    help='time features encoding, options:[timeF, fixed, learned]')
parser.add_argument('--activation', type=str, default='gelu', help='activation')
parser.add_argument('--output_attention', action='store_true', help='whether to output attention in ecoder')
parser.add_argument('--do_predict', action='store_true', help='whether to predict unseen future data')

# optimization
parser.add_argument('--num_workers', type=int, default=10, help='data loader num workers')
parser.add_argument('--itr', type=int, default=2, help='experiments times')
parser.add_argument('--train_epochs', type=int, default=100, help='train epochs')
parser.add_argument('--batch_size', type=int, default=100, help='batch size of train input data')
parser.add_argument('--patience', type=int, default=100, help='early stopping patience')
parser.add_argument('--learning_rate', type=float, default=0.0001, help='optimizer learning rate')
parser.add_argument('--des', type=str, default='test', help='exp description')
parser.add_argument('--loss', type=str, default='mse', help='loss function')
parser.add_argument('--lradj', type=str, default='type3', help='adjust learning rate')
parser.add_argument('--pct_start', type=float, default=0.3, help='pct_start')
parser.add_argument('--use_amp', action='store_true', help='use automatic mixed precision training', default=False)

# GPU
parser.add_argument('--use_gpu', type=bool, default=True, help='use gpu')
parser.add_argument('--gpu', type=int, default=0, help='gpu')
parser.add_argument('--use_multi_gpu', action='store_true', help='use multiple gpus', default=False)
parser.add_argument('--devices', type=str, default='0,1,2,3', help='device ids of multile gpus')
parser.add_argument('--test_flop', action='store_true', default=False, help='See utils/tools for usage')


args = parser.parse_args(args=[])

import random
# random seed
fix_seed = args.random_seed
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)


args.use_gpu = True if torch.cuda.is_available() and args.use_gpu else False

if args.use_gpu and args.use_multi_gpu:
    args.dvices = args.devices.replace(' ', '')
    device_ids = args.devices.split(',')
    args.device_ids = [int(id_) for id_ in device_ids]
    args.gpu = args.device_ids[0]

print('Args in experiment:')
print(args)

Args in experiment:
Namespace(random_seed=2021, root_path='./data/ETT/', data_path='ETTh1.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', seq_len=500, label_len=40, pred_len=60, fc_dropout=0.05, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=10, dec_in=10, c_out=10, d_model=512, n_heads=8, e_layers=2, d_layers=1, d_ff=2048, moving_avg=25, factor=1, distil=True, dropout=0.05, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=100, batch_size=100, patience=100, learning_rate=0.0001, des='test', loss='mse', lradj='type3', pct_start=0.3, use_amp=False, use_gpu=False, gpu=0, use_multi_gpu=False, devices='0,1,2,3', test_flop=False)


In [17]:
torch.backends.mps.is_available()

True

In [13]:
def train_one_epoch(args, s_dl):
    models = {'PatchTST': PatchTST, 'Informer': Informer}
    # models = {'PatchTST': PatchTST, 'Informer': Informer, 'Autoformer': Autoformer, 'DLinear': DLinear,
    #          'Transformer': Transformer}
    loss_fn = nn.MSELoss()
    device = torch.device('mps')
    for m in models.keys():
        args.model = models[m]
        model = args.model.Model(args).to(device)
        running_loss = 0.
        optimizer= optim.Adam(model.parameters(), lr=args.learning_rate)
        start = time.time()
        for i, data in enumerate(s_dl):
            # Every data instance is an input + label pair
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
        
            # Zero your gradients for every batch! 
            optimizer.zero_grad()
        
            # Make predictions for this batch
            dec_inp = torch.zeros_like(labels[:, -args.pred_len:, :]).float()
            dec_inp = torch.cat([labels[:, :args.label_len, :], dec_inp], dim=1).float()
        
            if 'Linear' in m or 'TST' in m:
                outputs = model(inputs)
            else: 
            # outputs = model(inputs)
                outputs = model(inputs, inputs[:,:,:4], dec_inp, labels[:,:,:4])
        
            outputs = outputs[:, -args.pred_len:, :]
            labels = labels[:, -args.pred_len:, :]
            
            # Compute the loss and its gradients
            loss = loss_fn(outputs, labels)
            loss.backward()
        
            # Adjust learning weights
            optimizer.step()
        
            # Gather data and report
            running_loss += loss.item()
        stop = time.time()
        avg_loss = running_loss / len(s_dl)
        print(m, ': ', avg_loss, ' | duration: ', stop-start)

PatchTST :  923.2179946899414  | duration:  56.7196569442749
Informer :  3104.5015970865884  | duration:  37.66055178642273


In [20]:

models = {'PatchTST': PatchTST}
# models = {'PatchTST': PatchTST, 'Informer': Informer}
# models = {'PatchTST': PatchTST, 'Informer': Informer, 'Autoformer': Autoformer, 'DLinear': DLinear,
#          'Transformer': Transformer}
loss_fn = nn.MSELoss()
device = torch.device('mps')

for m in models.keys():
    args.model = models[m]
    model = args.model.Model(args).to(device)
    running_loss = 0.
    optimizer= optim.Adam(model.parameters(), lr=args.learning_rate)
    start = time.time()
    for i, data in enumerate(s_dl):
        # Every data instance is an input + label pair
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
    
        # Zero your gradients for every batch! 
        optimizer.zero_grad()
    
        # Make predictions for this batch
        dec_inp = torch.zeros_like(labels[:, -args.pred_len:, :]).float()
        dec_inp = torch.cat([labels[:, :args.label_len, :], dec_inp], dim=1).float()
    
        if 'Linear' in m or 'TST' in m:
            outputs = model(inputs)
        else: 
        # outputs = model(inputs)
            outputs = model(inputs, inputs[:,:,:4], dec_inp, labels[:,:,:4])
    
        outputs = outputs[:, -args.pred_len:, :]
        labels = labels[:, -args.pred_len:, :]
        
        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)
        loss.backward()
    
        # Adjust learning weights
        optimizer.step()
    
        # Gather data and report
        running_loss += loss.item()
    stop = time.time()
    avg_loss = running_loss / len(s_dl)
    print(m, ': ', avg_loss, ' | duration: ', stop-start)

PatchTST :  925.5157216389974  | duration:  58.14438796043396


In [31]:
torch.mps.empty_cache()

In [11]:
# models = {'PatchTST': PatchTST, 'Informer': Informer, 'Autoformer': Autoformer, 'DLinear': DLinear,
#          'Transformer': Transformer}
models = {'PatchTST': PatchTST}
loss_fn = nn.MSELoss()
device = torch.device('mps')

for m in models.keys():
    args.model = models[m]
    model = args.model.Model(args).to(device)
    running_loss = 0.
    optimizer= optim.Adam(model.parameters(), lr=args.learning_rate)
    start = time.time()
    for i, data in enumerate(w_dl):
        # Every data instance is an input + label pair
        inputs, labels = data
        # inputs = inputs
        # labels = labels
    
        # Zero your gradients for every batch! 
        optimizer.zero_grad()
    
        # Make predictions for this batch
        dec_inp = torch.zeros_like(labels[:, -args.pred_len:, :]).float()
        dec_inp = torch.cat([labels[:, :args.label_len, :], dec_inp], dim=1).float()
    
        if 'Linear' in m or 'TST' in m:
            outputs = model(inputs)
        else: 
        # outputs = model(inputs)
            outputs = model(inputs, inputs[:,:,:4], dec_inp, labels[:,:,:4])
    
        outputs = outputs[:, -args.pred_len:, :]
        labels = labels[:, -args.pred_len:, :]
        
        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)
        loss.backward()
    
        # Adjust learning weights
        optimizer.step()
    
        # Gather data and report
        running_loss += loss.item()
    stop = time.time()
    avg_loss = running_loss / len(w_dl)
    print(m, ': ', avg_loss, ' | duration: ', stop-start)


PatchTST :  916.0164006551107  | duration:  58.74329495429993


In [ ]:
def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(training_loader):
        # Every data instance is an input + label pair
        inputs, labels = data

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        # if i % 1000 == 999:
        #     last_loss = running_loss / 1000 # loss per batch
        #     print('  batch {} loss: {}'.format(i + 1, last_loss))
        #     tb_x = epoch_index * len(training_loader) + i + 1
        #     tb_writer.add_scalar('Loss/train', last_loss, tb_x)
        #     running_loss = 0.
    avg_loss = running_loss / len(training_loader)

    return last_loss

In [408]:
from data_provider.data_loader import Dataset_ETT_hour as etth

In [410]:
root_path = '../dataset/'
# size = [params.context_points, 0, params.target_points]
# dls = DataLoaders(
#         datasetCls=Dataset_ETT_hour,
#         dataset_kwargs={
#         'root_path': root_path,
#         'data_path': 'ETTh2.csv',
#         'features': params.features,
#         'scale': True,
#         'size': size,
#         'use_time_features': False
#         },
#         batch_size=params.batch_size,
#         workers=params.num_workers,
#         )
d = etth(root_path=root_path)

In [412]:
len(d)

8161

In [414]:
len(d[0])

4

In [415]:
d[0][0].shape, d[0][1].shape, d[0][2].shape, d[0][3].shape

((384, 1), (192, 1), (384, 4), (192, 4))